In [1]:
import requests
import json
import pprint

- Get all shops

In [2]:
all_shops_API = requests.get("https://www.foody.vn/__get/Place/HomeListPlace?page=1&count=100&districtId=&cateId=&cuisineId=&type=1", 
                            headers={'X-Requested-With': 'XMLHttpRequest'})
data = all_shops_API.text
dataJson = json.loads(data)
print(dataJson)

{'CityId': 218, 'ExcludeIds': None, 'Query': None, 'Items': [{'Id': 639912, 'Name': 'Lẩu Cua Ngọc Hà', 'Address': '6 Chợ Ngọc Hà, P. Đội Cấn, Quận Ba Đình, Hà Nội', 'AvgRating': 7.6, 'AvgRatingText': '7.6', 'RestaurantStatus': 2, 'Phone': '0989 851 819', 'PhotoUrl': 'https://images.foody.vn/res/g64/639912/prof/s640x400/foody-mobile-31-jpg-544-636245918153020491.jpg', 'TotalReviews': 1, 'TotalFavourites': 0, 'TotalViews': 0, 'TotalPictures': 3, 'TotalCheckIns': 0, 'Url': '/ha-noi/lau-cua-ngoc-ha', 'Tags': None, 'Description': None, 'Note': {'RestaurantId': 0, 'UserId': 0, 'ListId': None, 'ListName': None, 'CreatedOn': '/Date(-62135596800000)/', 'Description': None, 'Id': 0}, 'HasPromotion': False, 'HasMemberCard': False, 'MemberCardDiscount': None, 'Latitude': 21.0334566034769, 'Longitude': 105.8311134204268, 'ResUsername': None, 'ResCreatedOn': '/Date(-62135596800000)/', 'LstPicture': [], 'LstReview': [{'reviewProfileUrl': '/thanh-vien/nongtrangminhvu', 'reviewUserDisplayName': 'Lẩu Ng

In [3]:
def extractshop(page, count):
    url = f'https://www.foody.vn/__get/Place/HomeListPlace?page={page}&count={count}&districtId=&cateId=&cuisineId=&type=1'
    all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest'})
    data = all_shops_API.text
    dataJson = json.loads(data)
    
    shop_review = []
    CityId = dataJson['CityId']
    
    for shop in dataJson['Items']: 
        shop_review.append({'CityId': CityId, 
                            'name': shop['Name'],
                            'addr': shop['Address'],
                            'RestaurantId': shop['RestaurantId'],
                            'RestaurantStatus': shop['RestaurantStatus'],
                            'avgrating' : shop['AvgRating'],
                            'TotalPictures' : shop['TotalPictures'],
                            'TotalSaves' : shop['TotalSaves'],
                            'TotalReviews' : shop['TotalReviews'],
                            'TotalFavourites' : shop['TotalFavourites'],
                            'TotalViews' : shop['TotalViews'],
                            'TotalCheckIns' : shop['TotalCheckIns'],
                            'CreatedOn'  : shop['ResCreatedOn'],
                            'Latitude'  : shop['Latitude'],
                            'Longitude'  : shop['Longitude'],
                            'ResUrlReviews' : shop['ResUrlReviews']})
    return shop_review

lmao = extractshop(1, 500)
lmao

[{'CityId': 218,
  'name': 'Lẩu Cua Ngọc Hà',
  'addr': '6 Chợ Ngọc Hà, P. Đội Cấn, Quận Ba Đình, Hà Nội',
  'RestaurantId': 639912,
  'RestaurantStatus': 2,
  'avgrating': 7.6,
  'TotalPictures': 3,
  'TotalSaves': 9,
  'TotalReviews': 1,
  'TotalFavourites': 0,
  'TotalViews': 0,
  'TotalCheckIns': 0,
  'CreatedOn': '/Date(-62135596800000)/',
  'Latitude': 21.0334566034769,
  'Longitude': 105.8311134204268,
  'ResUrlReviews': '/ha-noi/lau-cua-ngoc-ha/binh-luan'},
 {'CityId': 218,
  'name': 'YouOne Hotdog - Xúc Xích Hàn Quốc - Phan Văn Trường',
  'addr': '26H14 Ngõ 31 Phan Văn Trường, P. Dịch Vọng Hậu, Quận Cầu Giấy, Hà Nội',
  'RestaurantId': 686840,
  'RestaurantStatus': 2,
  'avgrating': 7.918,
  'TotalPictures': 214,
  'TotalSaves': 716,
  'TotalReviews': 73,
  'TotalFavourites': 0,
  'TotalViews': 0,
  'TotalCheckIns': 0,
  'CreatedOn': '/Date(-62135596800000)/',
  'Latitude': 21.038761,
  'Longitude': 105.785599,
  'ResUrlReviews': '/ha-noi/youone-hotdog-xuc-xich-han-quoc/binh-l

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [ ]:
# VD: có thể thay request_id = 1031704 để thử
delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

- get detail of ONE shop

In [40]:
delivery_dish_API = requests.get("https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={request_id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                            'x-foody-app-type': '1004',
                                            'x-foody-client-language': 'vi',
                                            'x-foody-client-type': '1',
                                            'x-foody-client-version': '1',
                                            'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
data = delivery_dish_API.text
dataJson = json.loads(data)
print(dataJson)

{'reply': {'delivery_detail': {'total_order': 0, 'rating': {'total_review': 500, 'avg': 4.7, 'display_total_review': '500+', 'app_link': 'foody://foody.vn?action=reviewlist&id=1031704'}, 'is_subscribe': False, 'is_favorite': False, 'city_id': 217, 'phones': ['0776969606'], 'restaurant_id': 1031704, 'district_id': 16, 'brand_id': 11607, 'video': None, 'asap_is_available': False, 'contract_type': 1, 'id': 121227, 'location_url': 'ho-chi-minh', 'foody_service_id': 1, 'is_quality_merchant': True, 'available_times': [{'date': '2022-09-21', 'times': ['09:55', '10:00', '10:05', '10:10', '10:15', '10:20', '10:25', '10:30', '10:35', '10:40', '10:45', '10:50', '10:55', '11:00', '11:05', '11:10', '11:15', '11:20', '11:25', '11:30', '11:35', '11:40', '11:45', '11:50', '11:55', '12:00', '12:05', '12:10', '12:15', '12:20', '12:25', '12:30', '12:35', '12:40', '12:45', '12:50', '12:55', '13:00', '13:05', '13:10', '13:15', '13:20', '13:25', '13:30', '13:35', '13:40', '13:45', '13:50', '13:55', '14:00',